In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

Cargar el dataset

In [39]:
data = pd.read_csv("Temp_Asu20092021.csv")

In [40]:
# Convertir la columna 'Fecha' a tipo datetime
data['Fecha'] = pd.to_datetime(data['Fecha'])

# Establecer la columna 'Fecha' como índice
data.set_index('Fecha', inplace=True)

# Filtrar datos desde 2019 en adelante
data = data[data.index.year >= 2019]

# Eliminar filas con valores faltantes
data.dropna(inplace=True)

# Resamplear el conjunto de datos a intervalos diarios
# Calcular la temperatura máxima diaria
daily_max_temperatures = data['Temperatura'].resample('D').max()

# Calcular la temperatura media diaria
daily_mean_temperatures = data['Temperatura'].resample('D').mean()

# Definir la variable objetivo (target) como la temperatura máxima desplazada
salida = daily_max_temperatures.shift(-1)

# Eliminar el último valor de las series debido al `shift(-1)`
daily_max_temperatures = daily_max_temperatures[:-1]
daily_mean_temperatures = daily_mean_temperatures[:-1]
salida = salida[:-1]

# Definir características (features): temperatura máxima y media diarias
X = np.column_stack([daily_max_temperatures.values, daily_mean_temperatures.values])
y = salida.values

# Normalizar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y validación
train_indices = daily_max_temperatures.index.year <= 2020
val_indices = daily_max_temperatures.index.year == 2021

X_train, X_val = X_scaled[train_indices], X_scaled[val_indices]
y_train, y_val = y[train_indices], y[val_indices]

# Definir el modelo de regresión Ridge
ridge = Ridge()

# Definir la cuadrícula de parámetros para GridSearch
param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]  # Regularización L2
}

# Usar GridSearchCV para buscar los mejores hiperparámetros
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:")
print(grid_search.best_params_)

# Evaluar el modelo con los mejores hiperparámetros en el conjunto de validación
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print(f"Mean Squared Error en validación: {mse}")
print(f"Root Mean Squared Error: {np.sqrt(mse)}")

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Mejores hiperparámetros:
{'alpha': 0.01}
Mean Squared Error en validación: 16.82617483703702
Root Mean Squared Error: 4.101972066827981
